### **Interactive Visual Analytics with Folium lab**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the exploratory data analysis part, we have visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between the launch site and success rates. This time we will be performing more interactive visual analytics using Folium.

#### **Objectives**

* TASK 1: Mark all launch sites on a map
* TASK 2: Mark the success/failed launches for each site on the map
* TASK 3: Calculate the distances between a launch site to its proximities

In [7]:
!pip install folium


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [9]:
spacex_df=pd.read_csv("spacex_launch_geo.csv")

Now, we can take a look at what are the coordinates for each site.

In [10]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


#### **Task 1: Mark All Launch Sites on A Map**

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.



In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [13]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Create and add folium.Circle and folium.Marker for each launch site on the site map

In [ ]:
# Add each launch site as a circle + labeled marker
for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site = row['Launch Site']
    # Circle
    folium.Circle(
        location=coord,
        radius=1000,
        color='#d35400',
        fill=True,
        fill_opacity=0.3
    ).add_child(folium.Popup(site)).add_to(site_map)

    # Text label marker
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size:12pt; color:#d35400;"><b>{site}</b></div>'
        )
    ).add_to(site_map)

site_map

#### **Task 2: Mark the Success/Failed Launches for Each Site on the Map**

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [18]:
marker_cluster = MarkerCluster()

In [19]:
# Create marker_color column: green for success (1), red for failure (0)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [20]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Loop through each launch record and add a marker to the cluster
for _, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=folium.Popup(f"{record['Launch Site']}<br>Success: {bool(record['class'])}", parse_html=True),
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

Click each launch site, from the color-labeled markers in marker clusters, we should be able to easily identify which launch sites have relatively high success rates.

#### **Task 3: Calculate the Distances Between a Launch Site to Its Proximities**

Next, we need to explore and analyze the proximities of launch sites. Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)

In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if we can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [23]:
# Define your coastline point (picked via MousePosition on the map)
coast_coord = [28.56367, -80.57163]

# Compute distance
site_row = launch_sites_df.iloc[0]
launch_coord = [site_row['Lat'], site_row['Long']]
distance_coastline = calculate_distance(launch_coord[0], launch_coord[1],coast_coord[0], coast_coord[1])

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

distance_marker = folium.Marker(
    location=coast_coord,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size:12pt; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

folium.PolyLine(locations=[launch_coord, coast_coord], color='blue', weight=2).add_to(site_map)

site_map

Similarly, we can draw a line betwee a launch site to its closest city, railway, highway, etc. We need to use MousePosition to find the their coordinates on the map first

In [28]:
# Define the launch site coordinate (reuse from before)
launch_coord = [site_row['Lat'], site_row['Long']]    # e.g. [28.56367, -80.57163]

# Pick the “closest city” coordinate by zooming in & reading MousePosition
city_coord = [28.60732, -80.80729]   # example: Titusville, FL

# Compute the distance
distance_city = calculate_distance(
    launch_coord[0], launch_coord[1],
    city_coord[0],  city_coord[1]
)

# Add a marker for the city with the distance label
city_marker = folium.Marker(
    location=city_coord,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size:12pt; color:blue;"><b>{distance_city:.2f} KM</b></div>'
    )
)
site_map.add_child(city_marker)

# Draw a line between the launch site and the city
folium.PolyLine(
    locations=[launch_coord, city_coord],
    color='blue',
    weight=2,
    opacity=0.7
).add_to(site_map)

# Repeat for railway, highway, etc., using their MousePosition coords
#  Display the updated map
site_map